## Stock Market Sentiment Analyzer

### **Project Overview**
This project aims to analyze sentiment in stock market-related tweets using Natural Language Processing (NLP). By leveraging **Hugging Face's FinBERT model**, we classify tweets as **positive, negative, or neutral** and explore how sentiment correlates with stock markettrends. 

In [9]:
# Important Necessary Libraries
import pandas as pd
import yfinance as yf 
import requests 
from bs4 import BeautifulSoup 
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
from datasets import load_dataset

# Load datasets 
dataset = load_dataset ("StephanAkkerman/stock-market-tweets-data")

print(['train'][0:5])

['train']


In [19]:
# Hugging Face Transformers for Sentiment Analysis
from transformers import pipeline

# Load FinBERT model
sentiment_pipeline = pipeline("text-classification", model="ProsusAI/finbert")

Device set to use mps:0


In [13]:
text = dataset['train'][0]['text']
result = sentiment_pipeline(text)
print(result)

[{'label': 'neutral', 'score': 0.536605179309845}]
